In [1]:
import opts
import argparse
import os
parser = argparse.ArgumentParser(description='Exploration.ipynb')
opts.translate_opts(parser)
opt = parser.parse_args(
    ["-split","pred","-output","pred.txt","-data_path","","-model_path","../data_model/wikisql/pretrain.pt"])

opt.anno = os.path.join(
    opt.data_path, 'annotated_ent/{}.jsonl'.format(opt.split))
opt.source_file = os.path.join(
    opt.data_path, 'data/{}.jsonl'.format(opt.split))
opt.db_file = os.path.join(opt.data_path, 'data/{}.db'.format(opt.split))
opt.pre_word_vecs = os.path.join(opt.data_path, 'embedding')


In [2]:
import torch
torch.cuda.set_device(opt.gpu)

In [3]:
dummy_parser = argparse.ArgumentParser(description='train.py')
opts.model_opts(dummy_parser)
opts.train_opts(dummy_parser)
dummy_opt = dummy_parser.parse_known_args([])[0]

In [ ]:
import table
fn_model = opt.model_path
opt.model=fn_model
translator = table.Translator(opt, dummy_opt.__dict__)

 22%|██▏       | 478104/2196017 [00:56<03:23, 8439.38it/s]

In [ ]:
import json
annotated_json_question='''{"id": "pred-0", "table_id": "1-1096793-1", "seq_output": {"gloss": ["SYMSELECT", "SYMAGG", "SYMCOL", "First", "season", "SYMWHERE", "SYMCOL", "Last", "title", "SYMOP", "=", "SYMCOND", "2012", "SYMEND"], "words": ["symselect", "symagg", "symcol", "first", "season", "symwhere", "symcol", "last", "title", "symop", "=", "symcond", "2012", "symend"], "after": [" ", "  ", " ", " ", " ", " ", " ", " ", " ", " ", " ", " ", " ", ""]}, "question": {"gloss": ["What", "was", "the", "first", "season", "of", "the", "club", "who", "last", "won", "a", "title", "in", "2012", "?"], "ent": ["WP", "VBD", "DT", "JJ", "NN", "IN", "DT", "NN", "WP", "JJ", "VBD", "DT", "NN", "IN", "CD", "."], "words": ["what", "was", "the", "first", "season", "of", "the", "club", "who", "last", "won", "a", "title", "in", "2012", "?"], "after": [" ", " ", " ", " ", " ", " ", " ", " ", " ", " ", " ", " ", " ", " ", "", ""]}, "query": {"sel": 2, "conds": [[6, 0, {"gloss": ["2012"], "words": ["2012"], "after": [""]}]], "agg": 0}, "table": {"header": [{"gloss": ["Club"], "words": ["club"], "after": [""]}, {"gloss": ["Position", "in", "2012"], "words": ["position", "in", "2012"], "after": [" ", " ", ""]}, {"gloss": ["First", "season"], "words": ["first", "season"], "after": [" ", ""]}, {"gloss": ["Number", "of", "seasons"], "words": ["number", "of", "seasons"], "after": [" ", " ", ""]}, {"gloss": ["First", "season", "of", "current", "spell"], "words": ["first", "season", "of", "current", "spell"], "after": [" ", " ", " ", " ", ""]}, {"gloss": ["Titles"], "words": ["titles"], "after": [""]}, {"gloss": ["Last", "title"], "words": ["last", "title"], "after": [" ", ""]}]}, "where_output": {"gloss": ["SYMWHERE", "SYMCOL", "Last", "title", "SYMOP", "=", "SYMCOND", "2012", "SYMEND"], "words": ["symwhere", "symcol", "last", "title", "symop", "=", "symcond", "2012", "symend"], "after": [" ", " ", " ", " ", " ", " ", " ", " ", ""]}, "seq_input": {"gloss": ["SYMSYMS", "SYMSELECT", "SYMWHERE", "SYMAND", "SYMCOL", "SYMTABLE", "SYMCAPTION", "SYMPAGE", "SYMSECTION", "SYMOP", "SYMCOND", "SYMQUESTION", "SYMAGG", "SYMAGGOPS", "SYMCONDOPS", "SYMAGGOPS", "MAX", "MIN", "COUNT", "SUM", "AVG", "SYMCONDOPS", "=", ">", "<", "OP", "SYMTABLE", "SYMCOL", "Club", "SYMCOL", "Position", "in", "2012", "SYMCOL", "First", "season", "SYMCOL", "Number", "of", "seasons", "SYMCOL", "First", "season", "of", "current", "spell", "SYMCOL", "Titles", "SYMCOL", "Last", "title", "SYMQUESTION", "What", "was", "the", "first", "season", "of", "the", "club", "who", "last", "won", "a", "title", "in", "2012", "?", "SYMEND"], "words": ["symsyms", "symselect", "symwhere", "symand", "symcol", "symtable", "symcaption", "sympage", "symsection", "symop", "symcond", "symquestion", "symagg", "symaggops", "symcondops", "symaggops", "max", "min", "count", "sum", "avg", "symcondops", "=", ">", "<", "op", "symtable", "symcol", "club", "symcol", "position", "in", "2012", "symcol", "first", "season", "symcol", "number", "of", "seasons", "symcol", "first", "season", "of", "current", "spell", "symcol", "titles", "symcol", "last", "title", "symquestion", "what", "was", "the", "first", "season", "of", "the", "club", "who", "last", "won", "a", "title", "in", "2012", "?", "symend"], "after": [" ", " ", " ", " ", " ", " ", " ", " ", " ", " ", " ", " ", " ", " ", " ", "  ", " ", " ", " ", " ", " ", " ", " ", " ", " ", " ", " ", " ", " ", " ", " ", " ", " ", " ", " ", " ", " ", " ", " ", " ", " ", " ", " ", " ", " ", " ", " ", " ", " ", " ", " ", " ", " ", " ", " ", " ", " ", " ", " ", " ", " ", " ", " ", " ", " ", " ", "", " ", ""]}}'''
annotated_question = json.loads(annotated_json_question)
js_list = [annotated_question]

In [ ]:
data = table.IO.TableDataset(js_list, translator.fields, None, False)

In [ ]:
test_data = table.IO.OrderedIterator(dataset=data,device=opt.gpu, batch_size=opt.batch_size, train=False, sort=True, sort_within_batch=False)
batch=next(iter(test_data))
result_list=translator.translate(batch)

In [ ]:
pred = result_list[0]
pred.predict(annotated_question)

In [ ]:
question = ["What", "was", "the", "first", "season", "of", "the", "club", "who", "last", "won", "a", "title", "in", "2012", "?"]
table = []


In [ ]:
table={"header": 
       ["Season", "Driver", "Team", "Engine", "Poles", "Wins", "Podiums", "Points", "Margin of defeat"],
       "page_title": "List of Formula One World Drivers' Championship runners-up", 
       "types": ["real", "text", "text", "text", "real", "real", "real", "text", "text"], 
       "id": "1-10753917-1",
       "section_title": "By season", 
       "caption": "By season",
       "rows": [[1950, "Juan Manuel Fangio", "Alfa Romeo", "Alfa Romeo", 4, 3, 3, "27", "3"], 
                #skip rows
                [2011, "Jenson Button", "McLaren", "Mercedes", 0, 3, 12, "270", "122"]], 
       "name": "table_10753917_1"
      }

In [ ]:
tables[table['id']]=table

In [ ]:
question = {"phase": 1, 
            "table_id": "1-10753917-1", 
            "question": "How many drivers on the williams team had a margin of defeat of 2?", 
            "sql": {"sel": 1, "conds": [[2, 0, "Williams"], [8, 0, "2"]], "agg": 3}
           }

In [ ]:
from annotate import annotate_example
annotated_question = annotate_example(question, table)
annotated_question